# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 18 2022 1:16PM,242102,19,ACG-2102490635,ACG North America LLC,Released
1,Jul 18 2022 12:53PM,242098,21,620594,"NBTY Global, Inc.",Released
2,Jul 18 2022 12:44PM,242097,21,620595,"NBTY Global, Inc.",Executing
3,Jul 18 2022 12:37PM,242095,10,8291462,"Snap Medical Industries, LLC",Released
4,Jul 18 2022 12:37PM,242095,10,8291463,"Snap Medical Industries, LLC",Released
5,Jul 18 2022 12:37PM,242095,10,8291464,"Snap Medical Industries, LLC",Released
6,Jul 18 2022 12:33PM,242094,10,EYE42970,Eye Pharma Inc,Released
7,Jul 18 2022 12:33PM,242093,21,402502719,"NBTY Global, Inc.",Released
8,Jul 18 2022 12:32PM,241980,10,PRF-31005,Bio-PRF,Released
9,Jul 18 2022 12:32PM,241980,10,PRF-31036,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,242094,Released,1
38,242095,Released,3
39,242097,Executing,1
40,242098,Released,1
41,242102,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242094,NaN,NaN,1.0
242095,NaN,NaN,3.0
242097,NaN,1.0,NaN
242098,NaN,NaN,1.0
242102,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241972,0.0,6.0,8.0
241980,0.0,0.0,31.0
242005,0.0,0.0,3.0
242007,0.0,0.0,1.0
242009,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241972,0,6,8
241980,0,0,31
242005,0,0,3
242007,0,0,1
242009,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241972,0,6,8
1,241980,0,0,31
2,242005,0,0,3
3,242007,0,0,1
4,242009,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241972,,6,8
1,241980,,,31
2,242005,,,3
3,242007,,,1
4,242009,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 18 2022 1:16PM,242102,19,ACG North America LLC
1,Jul 18 2022 12:53PM,242098,21,"NBTY Global, Inc."
2,Jul 18 2022 12:44PM,242097,21,"NBTY Global, Inc."
3,Jul 18 2022 12:37PM,242095,10,"Snap Medical Industries, LLC"
6,Jul 18 2022 12:33PM,242094,10,Eye Pharma Inc
7,Jul 18 2022 12:33PM,242093,21,"NBTY Global, Inc."
8,Jul 18 2022 12:32PM,241980,10,Bio-PRF
39,Jul 18 2022 12:32PM,242092,10,Eywa Pharma Inc.
40,Jul 18 2022 12:31PM,242091,21,"NBTY Global, Inc."
41,Jul 18 2022 12:28PM,242090,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 18 2022 1:16PM,242102,19,ACG North America LLC,,,1
1,Jul 18 2022 12:53PM,242098,21,"NBTY Global, Inc.",,,1
2,Jul 18 2022 12:44PM,242097,21,"NBTY Global, Inc.",,1,
3,Jul 18 2022 12:37PM,242095,10,"Snap Medical Industries, LLC",,,3
4,Jul 18 2022 12:33PM,242094,10,Eye Pharma Inc,,,1
5,Jul 18 2022 12:33PM,242093,21,"NBTY Global, Inc.",,,1
6,Jul 18 2022 12:32PM,241980,10,Bio-PRF,,,31
7,Jul 18 2022 12:32PM,242092,10,Eywa Pharma Inc.,,,1
8,Jul 18 2022 12:31PM,242091,21,"NBTY Global, Inc.",,,1
9,Jul 18 2022 12:28PM,242090,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 18 2022 1:16PM,242102,19,ACG North America LLC,1,,
1,Jul 18 2022 12:53PM,242098,21,"NBTY Global, Inc.",1,,
2,Jul 18 2022 12:44PM,242097,21,"NBTY Global, Inc.",,1,
3,Jul 18 2022 12:37PM,242095,10,"Snap Medical Industries, LLC",3,,
4,Jul 18 2022 12:33PM,242094,10,Eye Pharma Inc,1,,
5,Jul 18 2022 12:33PM,242093,21,"NBTY Global, Inc.",1,,
6,Jul 18 2022 12:32PM,241980,10,Bio-PRF,31,,
7,Jul 18 2022 12:32PM,242092,10,Eywa Pharma Inc.,1,,
8,Jul 18 2022 12:31PM,242091,21,"NBTY Global, Inc.",1,,
9,Jul 18 2022 12:28PM,242090,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 18 2022 1:16PM,242102,19,ACG North America LLC,1,,
1,Jul 18 2022 12:53PM,242098,21,"NBTY Global, Inc.",1,,
2,Jul 18 2022 12:44PM,242097,21,"NBTY Global, Inc.",,1,
3,Jul 18 2022 12:37PM,242095,10,"Snap Medical Industries, LLC",3,,
4,Jul 18 2022 12:33PM,242094,10,Eye Pharma Inc,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 18 2022 1:16PM,242102,19,ACG North America LLC,1.0,NaN,NaN
1,Jul 18 2022 12:53PM,242098,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jul 18 2022 12:44PM,242097,21,"NBTY Global, Inc.",NaN,1.0,NaN
3,Jul 18 2022 12:37PM,242095,10,"Snap Medical Industries, LLC",3.0,NaN,NaN
4,Jul 18 2022 12:33PM,242094,10,Eye Pharma Inc,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 18 2022 1:16PM,242102,19,ACG North America LLC,1.0,0.0,0.0
1,Jul 18 2022 12:53PM,242098,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jul 18 2022 12:44PM,242097,21,"NBTY Global, Inc.",0.0,1.0,0.0
3,Jul 18 2022 12:37PM,242095,10,"Snap Medical Industries, LLC",3.0,0.0,0.0
4,Jul 18 2022 12:33PM,242094,10,Eye Pharma Inc,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3388736,145.0,51.0,0.0
15,484119,11.0,12.0,3.0
16,242056,0.0,1.0,0.0
19,726259,3.0,0.0,0.0
20,1210116,27.0,12.0,0.0
21,2905042,10.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3388736,145.0,51.0,0.0
1,15,484119,11.0,12.0,3.0
2,16,242056,0.0,1.0,0.0
3,19,726259,3.0,0.0,0.0
4,20,1210116,27.0,12.0,0.0
5,21,2905042,10.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,145.0,51.0,0.0
1,15,11.0,12.0,3.0
2,16,0.0,1.0,0.0
3,19,3.0,0.0,0.0
4,20,27.0,12.0,0.0
5,21,10.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,145.0
1,15,Released,11.0
2,16,Released,0.0
3,19,Released,3.0
4,20,Released,27.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,0.0,3.0,0.0,0.0,0.0,0.0
Executing,51.0,12.0,1.0,0.0,12.0,2.0
Released,145.0,11.0,0.0,3.0,27.0,10.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,0.0,3.0,0.0,0.0,0.0,0.0
1,Executing,51.0,12.0,1.0,0.0,12.0,2.0
2,Released,145.0,11.0,0.0,3.0,27.0,10.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,0.0,3.0,0.0,0.0,0.0,0.0
1,Executing,51.0,12.0,1.0,0.0,12.0,2.0
2,Released,145.0,11.0,0.0,3.0,27.0,10.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()